In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from lxml import etree

# Función para obtener la URL de la página de resultados según el número de página
def get_search_url(page_number):
    base_url = 'https://listado.mercadolibre.com.ar/continental'
    if page_number > 1:
        return f'{base_url}_Desde_{(page_number - 1) * 50 + 1}_NoIndex_True'
    else:
        return base_url

# Lista para almacenar los DataFrames de cada página
all_dfs = []

# Número máximo de páginas a scrapear (ajústalo según sea necesario)
max_pages = 2

# Bucle para iterar sobre las páginas
for page_number in range(1, max_pages + 1):
    # Obtener la URL de la página actual
    current_url = get_search_url(page_number)

    # Realizar la solicitud para la página actual
    response = requests.get(current_url)

    # Verificar el código de estado
    if response.status_code == 200:
        print(f"Scraping data from page {page_number}")

        # Parsear la página actual
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tu código para extraer información de la página va aquí
        titulos = soup.find_all('h2', attrs={"class":"ui-search-item__title"})
        titulos = [i.text for i in titulos]

        urls = soup.find_all('a', attrs={"class":"ui-search-item__group__element ui-search-link__title-card ui-search-link"})
        urls = [i.get('href') for i in urls]

        dom = etree.HTML(str(soup))
        precios = dom.xpath('//li[contains(@class, "ui-search-layout__item")]//span[@class="andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript"]/span[@class="andes-money-amount__fraction"]/text()')

        # Crear un DataFrame para la página actual
        df = pd.DataFrame({
            'Cubierta': titulos,
            'Enlace': urls,
            'Precios': precios
        })

        # Agregar columnas adicionales
        df['Vendedor'] = ""
        df['Detalles'] = ""
        df['Cantidad'] = ""
        df['Estado'] = ""
        df['Categoria'] = ""


        # Iterar sobre los enlaces y obtener información adicional
        for index, row in df.iterrows():
            product_url = row['Enlace']
            product_response = requests.get(product_url)
            product_soup = BeautifulSoup(product_response.content, 'html.parser')

            # Obtener el color y precio
            vendedor = product_soup.find('span', attrs={"class": "ui-pdp-color--BLUE ui-pdp-family--REGULAR"})
            detalles = product_soup.find('span', attrs={"class": "ui-pdp-subtitle"})
            cantidad = product_soup.find('span', attrs={"class":"ui-pdp-buybox__quantity__available"})
            estado = product_soup.find('span', attrs={"class":"ui-pdp-subtitle"})
            categoria = product_soup.find('a', attrs={"class":"andes-breadcrumb__link"})


            # Asignar los valores a las columnas correspondientes
            df.at[index, 'Vendedor'] = vendedor.text if vendedor else "No disponible"
            df.at[index, 'Detalles'] = detalles.text if detalles else "No disponible"
            df.at[index, 'Cantidad'] = cantidad.text if cantidad else "No disponible"
            df.at[index, 'Estado'] = estado.text if estado else "No disponible"
            df.at[index, 'Categoria'] = categoria.text if categoria else "No disponible"



        # Agregar el DataFrame al listado
        all_dfs.append(df)

    else:
        print(f"Failed to retrieve data from page {page_number}. Status code: {response.status_code}")

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(all_dfs, ignore_index=True)

# Guardar en un archivo CSV
final_df.to_csv('smart_all_pages.csv', index=False)

print("Scraping completado. El archivo fue exportado con éxito.")


Scraping data from page 1
Scraping data from page 2
Scraping completado. El archivo fue exportado con éxito.


In [11]:
final_df

,Cubierta,Enlace,Precios,Vendedor,Detalles,Cantidad,Estado,Categoria,Categoria1
0,Neumático Continental ContiCrossContact LX 2 L...,https://www.mercadolibre.com.ar/neumatico-cont...,372.781,CALZETTA NEUMATICOS,Nuevo | +500 vendidos,(4 disponibles),Nuevo | +500 vendidos,Accesorios para Vehículos,Accesorios para Vehículos
1,Cubierta Continental Race King 27.5 X 2.2 Spor...,https://articulo.mercadolibre.com.ar/MLA-13683...,59.900,BOHEMIA.BIKES,Nuevo | +5 vendidos,(15 disponibles),Nuevo | +5 vendidos,Deportes y Fitness,Deportes y Fitness
2,Neumático Continental ContiEcoContact 3 165/70...,https://www.mercadolibre.com.ar/neumatico-cont...,192.739,CALZETTA NEUMATICOS,Nuevo | +500 vendidos,(4 disponibles),Nuevo | +500 vendidos,Accesorios para Vehículos,Accesorios para Vehículos
3,Neumático Continental ContiPowerContact P 205/...,https://www.mercadolibre.com.ar/neumatico-cont...,356.659,NEUMATICOSNORT,Nuevo | +500 vendidos,No disponible,Nuevo | +500 vendidos,Accesorios para Vehículos,Accesorios para Vehículos
4,Neumático Continental ContiPowerContact P 185/...,https://www.mercadolibre.com.ar/neumatico-cont...,254.999,ADIRUEDAS,Nuevo | +500 vendidos,(2 disponibles),Nuevo | +500 vendidos,Accesorios para Vehículos,Accesorios para Vehículos
...,...,...,...,...,...,...,...,...,...
104,Neumático Continental 205 55 16 91v Powercont...,https://articulo.mercadolibre.com.ar/MLA-11250...,246.050,NEUMAFLORES S.R.L.,Nuevo | +5 vendidos,(5 disponibles),Nuevo | +5 vendidos,Accesorios para Vehículos,Accesorios para Vehículos
105,Neumático Continental VanContact AP 225/70R15 ...,https://www.mercadolibre.com.ar/neumatico-cont...,433.096,CALZETTA NEUMATICOS,Nuevo | 3 vendidos,(4 disponibles),Nuevo | 3 vendidos,Accesorios para Vehículos,Accesorios para Vehículos
106,Cubiertas Ultra Sport,https://articulo.mercadolibre.com.ar/MLA-11212...,62.500,JUANA.QUEVEDO,Nuevo | +25 vendidos,(6 disponibles),Nuevo | +25 vendidos,Deportes y Fitness,Deportes y Fitness
107,Kit de 4 neumáticos Continental PowerContact 2...,https://www.mercadolibre.com.ar/kit-de-4-neuma...,1.027.210,CALZETTA NEUMATICOS,Nuevo | +5 vendidos,No disponible,Nuevo | +5 vendidos,Accesorios para Vehículos,Accesorios para Vehículos
